<a href="https://colab.research.google.com/github/SourabhThanvi/ZeoTap-Data-Science-Assignment-/blob/main/Sourabh_Thanvi_Customer_Segmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.cluster import KMeans
from sklearn.metrics import davies_bouldin_score
from sklearn.decomposition import PCA

In [2]:
cos_df = pd.read_csv('/content/Customers.csv')
trans_df = pd.read_csv('/content/Transactions.csv')

In [4]:
df = pd.merge(trans_df, cos_df, how = 'inner', on = 'CustomerID')

In [7]:
df

,TransactionID,CustomerID,ProductID,TransactionDate,Quantity,TotalValue,Price,CustomerName,Region,SignupDate
0,T00001,C0199,P067,2024-08-25 12:38:23,1,300.68,300.68,Andrea Jenkins,Europe,2022-12-03
1,T00112,C0146,P067,2024-05-27 22:23:54,1,300.68,300.68,Brittany Harvey,Asia,2024-09-04
2,T00166,C0127,P067,2024-04-25 07:38:55,1,300.68,300.68,Kathryn Stevens,Europe,2024-04-04
3,T00272,C0087,P067,2024-03-26 22:55:37,2,601.36,300.68,Travis Campbell,South America,2024-04-11
4,T00363,C0070,P067,2024-03-21 15:10:10,3,902.04,300.68,Timothy Perez,Europe,2022-03-15
...,...,...,...,...,...,...,...,...,...,...
995,T00496,C0118,P037,2024-10-24 08:30:27,1,459.86,459.86,Jacob Holt,South America,2022-01-22
996,T00759,C0059,P037,2024-06-04 02:15:24,3,1379.58,459.86,Mrs. Kimberly Wright,North America,2024-04-07
997,T00922,C0018,P037,2024-04-05 13:05:32,4,1839.44,459.86,Tyler Haynes,North America,2024-09-21
998,T00959,C0115,P037,2024-09-29 10:16:02,2,919.72,459.86,Joshua Hamilton,Asia,2024-11-11


In [8]:
aggregated_data = df.groupby('CustomerID').agg({
    'Quantity': 'sum',
    'TotalValue': 'sum',
    'ProductID': 'nunique',
    'Region': 'first'
}).reset_index()

In [9]:
aggregated_data

,CustomerID,Quantity,TotalValue,ProductID,Region
0,C0001,12,3354.52,5,South America
1,C0002,10,1862.74,4,Asia
2,C0003,14,2725.38,4,South America
3,C0004,23,5354.88,8,South America
4,C0005,7,2034.24,3,Asia
...,...,...,...,...,...
194,C0196,12,4982.88,3,Europe
195,C0197,9,1928.65,3,Europe
196,C0198,3,931.83,2,Europe
197,C0199,9,1979.28,4,Europe


In [11]:
scaler = StandardScaler()
numerical_features = ['Quantity', 'TotalValue', 'ProductID']
aggregated_data[numerical_features] = scaler.fit_transform(aggregated_data[numerical_features])

In [15]:
encoder = OneHotEncoder()
categorical_features = ['Region']
encoded_cats = encoder.fit_transform(aggregated_data[categorical_features]).toarray()

In [20]:
features = pd.concat([pd.DataFrame(aggregated_data[numerical_features]), pd.DataFrame(encoded_cats)], axis=1)
features.columns = features.columns.astype(str)

In [26]:
db_scores = []
silhouette_scores = []
cluster_range = range(2, 11)  # Testing 3 to 8 clusters

for k in cluster_range:
    # K-Means Clustering
    kmeans = KMeans(n_clusters=k, random_state=42)
    labels = kmeans.fit_predict(features)

    # Evaluate metrics
    db_index = davies_bouldin_score(features, labels)
    db_scores.append(db_index)

    print(f"k={k}, DB Index={db_index:.4f}")

k=2, DB Index=0.9980
k=3, DB Index=1.2033
k=4, DB Index=1.3151
k=5, DB Index=1.3571
k=6, DB Index=1.3236
k=7, DB Index=1.3620
k=8, DB Index=1.2599
k=9, DB Index=1.1199
k=10, DB Index=1.0631


In [27]:
kmeans = KMeans(n_clusters=2, random_state=42)
labels = kmeans.fit_predict(features)

In [28]:
db_index = davies_bouldin_score(features, labels)
db_index

0.99803032957982